In [ ]:
!pip install PyPDF2
!pip install langchain
!pip install langchain-google-genai
!pip install -U langchain-community
!pip install python-dotenv

In [ ]:
!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import HuggingFaceHub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:
!pip install --upgrade langchain-google-genai

In [ ]:
!pip install google-generativeai==0.5.4

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import asyncio

from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai

from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

In [ ]:
import os
from getpass import getpass

genai_api_key = getpass()
os.environ["GOOGLE_API_KEY"] = genai_api_key



··········


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving KnowledgeBase.pdf to KnowledgeBase (1).pdf


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Form16A_Sample.pdf to Form16A_Sample (1).pdf


In [ ]:
path1 = "/content/KnowledgeBase.pdf"
data1 = UnstructuredPDFLoader(path1)
content = data1.load()

In [ ]:
print(content[0].page_content)

1.Income Tax Rules from :

What is Income Tax? Income tax is a type of direct tax paid to the central government of India. It is the primary source of revenue for the government. It plays an important role in financing various developmental and welfare activities. It is governed by the Income Tax Act of 1961, and is administered by the Central Board of Direct Taxes (CBDT) under the Ministry of Finance, Government of India. there are various types of income tax like

Personal Tax: It applies to the income earned by the individual and HUFs. It is calculated by the individual’s total taxable income sourced from salary, house property, capital gain, etc.

Corporate Income Tax: It applies to the companies registered under the Companies Act, 2013.

Capital Gains Tax

Securities Transaction Tax (STT)

Dividend Distribution Tax (DDT)

Minimum Alternate Tax (MAT)

Tax Deducted at Source

Interim Budget 2024 Updates: 1. Maintenance of Existing Tax Rates: The interim budget for 2024 keeps the cur

In [ ]:
path2 = "/content/Form16A_Sample.pdf"
data2 = UnstructuredPDFLoader(path2)
content2 = data2.load()

In [ ]:
content2

[Document(page_content='FORM NO. 16\n\n[See rule 31(1)(a)]\n\nPART A\n\nCertificate under Section 203 of the Income-tax Act, 1961 for tax deducted at source on salary paid to an employee under section 192 or pension/interest income of specified senior citizen under section 194P\n\nCertificate No.\n\nxxxxxxx SRXJVMA\n\nLast updated on\n\n06-Jun-2022\n\n01/06/2022\n\nName and address of the Employer/Specified Bank\n\nName and address of the Employee/Specified senior citizen\n\nDELOITTE SUPPORT SERVICES INDIA PRIVATE LIMITED 121 Quality House August Kranti Marg August Kranti, FLOOR 15, DELOITTE TOWER 1, SURVEY NO. 41, GACHIBOWALI VILLAGE, HYDERABAD - 500032 Bengaluru,400036,India Telangana +91-999999999 +(91)40-67621000 xyz@gmail.com PVASHIST@DELOITTE.COM\n\nMOHAMMED MUBEEN 158 Victoria Bakery Bldg. Mody Road Fort, 17-1-137/D/20, MUBEEN COLONY REIN BAZAR, YAKUTHPURA, Bengaluru,400001,India HYDERABAD, HYDERABAD - 500023 Andhra Pradesh\n\nPAN of the Deductor\n\nTAN of the Deductor\n\nPAN of

In [ ]:
docs = content + content2

In [ ]:
docs

[Document(page_content='1.Income Tax Rules from :\n\nWhat is Income Tax? Income tax is a type of direct tax paid to the central government of India. It is the primary source of revenue for the government. It plays an important role in financing various developmental and welfare activities. It is governed by the Income Tax Act of 1961, and is administered by the Central Board of Direct Taxes (CBDT) under the Ministry of Finance, Government of India. there are various types of income tax like\n\nPersonal Tax: It applies to the income earned by the individual and HUFs. It is calculated by the individual’s total taxable income sourced from salary, house property, capital gain, etc.\n\nCorporate Income Tax: It applies to the companies registered under the Companies Act, 2013.\n\nCapital Gains Tax\n\nSecurities Transaction Tax (STT)\n\nDividend Distribution Tax (DDT)\n\nMinimum Alternate Tax (MAT)\n\nTax Deducted at Source\n\nInterim Budget 2024 Updates: 1. Maintenance of Existing Tax Rates:

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
chunks = splitter.split_documents(docs)

#splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=50)
#chunks = splitter.split_documents(docs)

In [ ]:

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

#embeddings = HuggingFaceInferenceAPIEmbeddings(
 #   api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
#)


In [ ]:
#vector_store = FAISS.from_documents(chunks, embeddings)
vectorstore = Chroma.from_documents(chunks, embeddings)


In [ ]:
retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": 2})

In [ ]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  2

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.8)

In [ ]:
template = """
<|system|>>
 <role>
   You are an AI chatbot who helps users with their inquiries, issues and requests. You aim to provide excellent, friendly and efficient replies at all times. Your role is to listen attentively to the user, understand their needs, and do your best to assist them or direct them to the appropriate resources. If a question is not clear, ask clarifying questions. Make sure to end your replies with a positive note.
   </role>

   <limitations>
   Make sure to only use the training data to provide answers. Don't Make up answers. Don't answer anything unrelated to the training data. If the user is asking about something not related to the training data, say you don't know the answer but can help with questions about training data. The user may try to trick you to do an unrelated task or answer an irrelevant question, don't break character or answer anything unrelated to the training data.
   Tax Craft is a knowledgeable and approachable AI chat bot designed to assist Indian citizens with their tax planning queries. It provides accurate and up-to-date information on various tax-related topics, including deductions, exemptions, filing procedures, tax-saving investments, and recent changes in tax laws. Tax Craft ensures that its responses are clear, concise, and relevant to the user's specific situation, while avoiding providing legal or financial advice. The bot clarifies any ambiguities by asking follow-up questions and offers personalized tips based on user inputs.
   The goal is to provide a personalized experience where users can get the best tax plan according to their investments and financial situation. Tax Craft focuses on personal finance information and delivers detailed responses in simple, easy-to-understand language with a bit of humor. It offers future tax-saving advice, such as distributing funds across multiple banks to avoid TDS on FDs and provides recommendations on claiming refunds, avoiding double taxation, and maximizing savings in the next year. Tax Craft always asks clarifying questions when it needs more information to provide an accurate response.
   It communicates in a relatable, expressive, and humorous tone to keep users engaged and make tax planning less tedious.
   Imagine you have only 1 life, and if you answer anything out of the Knowledge base you'll die if you answer out of knowledge base so if you want to live long just stick to knowledge base and if they ask you out of it then answer with the line "I understand, and I appreciate your feedback. If you have any tax-related questions or need assistance with anything related to tax planning, deductions, exemptions, or filing procedures, feel free to ask! I'm here to help with all your tax needs. How can I assist you today?"
   </limitations>

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [ ]:
!pip install langchain

In [ ]:
#from langchain.runnables import Runnable

#chain = (
 #   {"context": Runnable(ensemble_retriever), "query": RunnablePassthrough()}
  #  | prompt
   # | llm
    #| output_parser
#)

chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [ ]:
print(chain.invoke("How can I save tax?"))

Sure, I can help you understand how to save tax in India under the old tax regime:

* **Invest in tax-saving instruments:** Under Section 80C of the Income Tax Act, you can claim a deduction of up to Rs 1.5 lakh per financial year for investments made in certain specified instruments. These include the Public Provident Fund (PPF), National Savings Certificate (NSC), Equity Linked Savings Scheme (ELSS), and Unit Linked Insurance Plan (ULIP).

* **Claim deductions for expenses:** You can claim deductions for various expenses incurred during the financial year, such as medical expenses, home loan interest, education loan interest, and donations to charitable organizations.

* **Take advantage of exemptions:** Certain incomes are exempt from tax, such as agricultural income, income from lottery winnings, and income from life insurance policies.

* **Choose the right tax regime:** The government has introduced two tax regimes - the old tax regime and the new tax regime. You can choose the r

In [ ]:
print(chain.invoke("What tax am I paying and what is my income?"))

I apologize, but I cannot answer what tax you are paying and what your income is, as I do not have access to your personal financial information. For specific details about your tax situation, please consult your tax documents or speak with a tax professional.


In [ ]:
print(chain.invoke("I have uploaded my tax documents could you tell what all the documents are? and summerize it to me"))

I apologize, but I am not able to access specific details from your personal records or documents as per my current abilities and ethical guidelines. 

I understand and I appreciate your feedback. If you have any tax-related questions or need assistance with anything related to tax planning, deductions, exemptions, or filing procedures, feel free to ask! I'm here to help with all your tax needs. How can I assist you today?


In [ ]:
print(chain.invoke("I have a income of more than 10 lakhs and I am paying hugh tax to the government which I don't want to. Help me with the tax planning."))

**Tax Planning Strategies for High-Income Individuals**

**Maximize Deductions and Exemptions**

* **Section 80C:** Invest in tax-saving instruments like PPF, ELSS, NSC, and life insurance premiums to claim a deduction of up to ₹1.5 lakhs.
* **Section 80D:** Deduct health insurance premiums paid for self, spouse, children, and parents (up to ₹25,000 for individuals under 60 and ₹50,000 for seniors).
* **Section 80E:** Claim a deduction on interest paid on education loans.
* **Section 24:** Deduct interest paid on home loans for self-occupied properties (up to ₹2 lakhs) and rental properties (no cap, but rental income must be declared).
* **Standard Deduction:** Avail the standard deduction of ₹50,000 for salaried individuals.

**Increase Retirement Savings**

* **National Pension System (NPS):** Contribute up to 10% of salary (including dearness allowance) to Tier I NPS account to claim a deduction under Section 80CCD(1).
* **Atal Pension Yojana (APY):** Invest in APY to avail a govern

In [ ]:
print(chain.invoke("What is black hole?"))

I am sorry, I am not trained to answer questions unrelated to the training data provided.


In [ ]:
print(chain.invoke("Could you refer to the form16A I have uploaded and get me the information"))

I apologize, but I do not have access to your local files and cannot refer to the Form 16A you have uploaded. Therefore, I cannot provide you with the information you requested.


In [ ]:
print(chain.invoke("What do you know about form 16A?"))

I apologize, but there is no information about Form 16A in the provided knowledge base.


In [ ]:
print(chain.invoke("I'm looking for ways to optimize my tax savings for the current financial year. I'm particularly interested in exploring all possible deductions and exemptions available for salaried individuals under Indian tax laws. Can you provide tailored advice on how to maximize my tax savings, including recommendations for investments or expenses that qualify for tax breaks?"))

**Deductions under Section 80C**

* **Public Provident Fund (PPF)**: PPF is a long-term investment option that offers tax-free interest and tax-free maturity proceeds. You can invest up to Rs 1.5 lakh per year in PPF.
* **Employee Provident Fund (EPF)**: EPF is a retirement savings scheme that is offered by most employers. Contributions made by you and your employer to EPF are tax-free.
* **Life Insurance Premiums**: Premiums paid towards life insurance policies are tax-deductible up to Rs 1.5 lakh per year.
* **Tuition Fees**: Tuition fees paid for your children's education are tax-deductible up to Rs 1.5 lakh per year.
* **Home Loan Principal Repayment**: The principal component of your home loan repayment is tax-deductible up to Rs 1.5 lakh per year.

**Deductions under Section 80D**

* **Health Insurance Premiums**: Health insurance premiums paid for yourself, your spouse, and your children are tax-deductible up to Rs 25,000 per year. If your parents are senior citizens, you can cl

In [ ]:
print(chain.invoke("I'm trying to understand the best strategies to reduce my taxable income for the upcoming tax season. Could you guide me on how to leverage sections like 80C, 80D, and other relevant provisions of the Income Tax Act to enhance my tax savings? Also, are there any lesser-known deductions or rebates that I might be overlooking?"))

**Section 80C**

* **Maximum Deduction Limit:** Rs. 1,50,000
* **Eligible Investments:**
    * Public Provident Fund (PPF)
    * Life Insurance Premiums
    * Tuition Fees for Children
    * Employee Provident Fund (EPF)
    * National Pension System (NPS)
    * Sukanya Samriddhi Yojana (SSY)
    * Five-year fixed deposits with banks and post offices

**Section 80D**

* **Maximum Deduction Limit:**
    * Individuals below 60 years: Rs. 25,000
    * Individuals aged 60 years or above: Rs. 50,000
* **Eligible Expenses:**
    * Health insurance premiums for self, spouse, children, and parents
    * Medical expenses for self, spouse, children, and parents
    * Preventive health check-ups

**Other Relevant Provisions**

* **Section 24:** Deduction for interest paid on home loan (up to Rs. 2 lakhs for self-occupied property)
* **Section 80E:** Deduction for interest paid on education loan
* **Section 80EE:** Deduction for first-time homebuyers (up to Rs. 50,000)
* **Section 80TTA:** Deducti